In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import os


In [52]:
class LinearReg:
    def __init__(self, alpha=0.1):
        self.weights = None
        self.bias = None
        self.alpha = alpha

    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        n_samples, n_features = X_b.shape
        identity_matrix = np.eye(n_features)
        identity_matrix[0, 0] = 0

        theta = (
            np.linalg.inv(X_b.T.dot(X_b) + self.alpha * identity_matrix)
            .dot(X_b.T)
            .dot(y)
        )
        self.bias = theta[0]
        self.weights = theta[1:]

    def predict(self, X):
        return X.dot(self.weights) + self.bias

In [53]:
# Feature Engineering Functions
def create_interaction_features(X):
    """Create interaction features between pairs of features"""
    feature_names = X.columns
    X_array = X.values
    n_features = X_array.shape[1]
    interactions = []
    new_feature_names = list(feature_names)
    
    for i in range(n_features):
        for j in range(i+1, n_features):
            interaction = X_array[:, i] * X_array[:, j]
            interactions.append(interaction)
            new_feature_names.append(f"{feature_names[i]}_{feature_names[j]}")
    
    interactions_array = np.column_stack(interactions)
    X_with_interactions = np.column_stack([X_array, interactions_array])
    return pd.DataFrame(X_with_interactions, columns=new_feature_names)


def add_polynomial_features(X, degree=2):
    """Add polynomial features up to specified degree"""
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_features = poly.fit_transform(X)

    # Generate feature names for polynomial features
    feature_names = poly.get_feature_names_out(X.columns)

    return pd.DataFrame(poly_features, columns=feature_names)

In [54]:
# Outlier Detection and Handling
def remove_outliers(X, y, threshold=2.5):  # Reduced threshold
    """Remove outliers based on z-score with more conservative threshold"""
    scaler = RobustScaler()  # Using RobustScaler instead of StandardScaler
    z_scores = np.abs(scaler.fit_transform(X))
    mask = (z_scores < threshold).all(axis=1)
    return X[mask].reset_index(drop=True), y[mask].reset_index(drop=True)

In [29]:
#Define Evaluation Metric Functions
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def r_squared(y_true, y_pred):
    y_mean = np.mean(y_true)
    total_variance = np.sum((y_true - y_mean) ** 2)
    explained_variance = np.sum((y_true - y_pred) ** 2)
    return 1 - (explained_variance / total_variance)

In [57]:
file_path = "./dataset/diabetes.csv"
if not os.path.exists(file_path):
    print('downloading Dataset...')
    iris = load_diabetes()
    store_iris = pd.DataFrame(
        data=np.c_[iris["data"], iris["target"]],
        columns=iris["feature_names"] + ["target"],
    ).to_csv(file_path,index=None)
    


df = pd.read_csv(file_path)

df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


In [33]:
df.isnull().sum()

age       0
sex       0
bmi       0
bp        0
s1        0
s2        0
s3        0
s4        0
s5        0
s6        0
target    0
dtype: int64

In [58]:
X = df.drop(['target'],axis = 1)
y = df['target']
print("Original shape:", X.shape)
X_clean, y_clean = remove_outliers(X, y, threshold=2.5)
print("Shape after removing outliers:", X_clean.shape)

#split dataset
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

Original shape: (442, 10)
Shape after removing outliers: (434, 10)


In [59]:
#interaction features
X_train_interactions = create_interaction_features(X_train)
X_test_interactions = create_interaction_features(X_test)

In [ ]:
# Add polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_interactions)
X_test_poly = poly.transform(X_test_interactions)

In [61]:
#feature scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

In [62]:
models = {
    'Lasso': LassoCV(cv=5, random_state=42, max_iter=2000),
    'Ridge': RidgeCV(cv=5),
    'ElasticNet': ElasticNetCV(cv=5, random_state=42, max_iter=2000)
}

In [63]:
results = {}
for name, model in models.items():
    # Fit model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    
    results[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'CV R² (mean)': cv_scores.mean(),
        'CV R² (std)': cv_scores.std()
    }

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.950e+02, tolerance: 1.678e+02
  model = cd_fast.enet_coordinate_descent(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.616e+02, tolerance: 1.678e+02
  model = cd_fast.enet_coordinate_descent(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale 

KeyboardInterrupt: 

In [ ]:
print("\nModel Comparison:")
for name, metrics in results.items():
    print(f"\n{name} Results:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.3f}")



Model Evaluation Metrics:
RMSE: 54.40
MAE: 44.96
R²: 0.36


In [ ]:
# Select the best model based on CV score
best_model_name = max(results, key=lambda k: results[k]['CV R² (mean)'])
best_model = models[best_model_name]

print(f"\nBest Model: {best_model_name}")

In [ ]:
# Feature importance analysis
if hasattr(best_model, 'coef_'):
    feature_importance = pd.DataFrame({
        'Feature': poly.get_feature_names_out(),
        'Coefficient': np.abs(best_model.coef_)
    })
    feature_importance = feature_importance.sort_values('Coefficient', ascending=False)
    
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10))


Top 5 Most Important Features:
       Feature       Score
3  sex sex_bmi  201.713491
8    sex^2 bmi  201.713491
0          bmi  192.184941
2           s5  155.497857
5   sex sex_s5  154.829482
